# Bayesian biodiversity: Geodata processing

In [10]:
import numpy as np
import pygeoprocessing as pgp
import geopandas as gpd
import pandas as pd
import rasterio
import shapely
from shapely import Point, Polygon
import time
from rasterstats import zonal_stats
from pyproj import Transformer

In [12]:
# Load black for formatting
import jupyter_black

jupyter_black.load()

# Adjust display settings for pandas
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [31]:
def project_to_utm(point):

    # Get the coordinate point values
    long, lat = point.x, point.y

    # Determine the UTM zone and hemisphere
    zone_number = int((long + 180) // 6) + 1
    epsg_code = f"EPSG:{32700 + zone_number if lat < 0 else 32600 + zone_number}"

    # Initialize Transformer object, and transform to UTM coordinates
    utm_transformer = Transformer.from_crs("EPSG:4326", epsg_code, always_xy=True)
    utm_x, utm_y = utm_transformer.transform(long, lat)
    utm_coords = shapely.Point(utm_x, utm_y)

    return utm_coords, epsg_code

In [32]:
def buffer_points_in_utm(points, buffer_dist, polygon_type="square"):

    # Buffer array of Points into the chosen size and type
    utm_coords_buff = shapely.buffer(points, buffer_dist, cap_style=polygon_type)

    return utm_coords_buff

In [78]:
def reproject_to_global(polygon, epsg_code):

    # Transformer for reprojecting to global coordinate system
    global_transformer = Transformer.from_crs(epsg_code, "EPSG:4326", always_xy=True)

    # Get the points that define the polygon as an array
    xx, yy = polygon.exterior.coords.xy
    x = np.array(xx)
    y = np.array(yy)
    coord_tuples = np.array([shapely.Point(xy) for xy in zip(x, y)])

    # Perform the reprojection
    projected_coords = [
        global_transformer.transform(coord.x, coord.y) for coord in coord_tuples
    ]
    global_polygon = shapely.Polygon(projected_coords)

    return global_polygon

In [92]:
def project_buffer_reproject_sites(gdf_sites, buffer_distances):

    # Make a copy of the original dataframe
    gdf_sites = gdf_sites.copy()

    # Rename geometry for clarity, since there are multiple ones added
    gdf_sites = gdf_sites.rename(columns={"geometry": "global_coord"})

    # Project each point to UTM
    gdf_sites[["utm_coord", "epsg_code"]] = gdf_sites.apply(
        lambda row: project_to_utm(row["global_coord"]),
        axis=1,
        result_type="expand",
    )

    # Buffer different sized polygons and append as new columns
    for buffer_dist in buffer_distances:
        gdf_sites[f"utm_{buffer_dist}_km"] = buffer_points_in_utm(
            gdf_sites["utm_coord"], buffer_dist * 1000
        )

    # Reproject the polygons to global coordinate format
    for buffer_dist in buffer_distances:
        gdf_sites[f"glob_{buffer_dist}_km"] = gdf_sites.apply(
            lambda row: reproject_to_global(
                row[f"utm_{buffer_dist}_km"], row["epsg_code"]
            ),
            axis=1,
        )

    return gdf_sites

In [93]:
def save_buffered_site_files(gdf_sites, buffer_distances):

    for buffer_dist in buffer_distances:
        gdf_res = gdf_sites[["SSS", f"glob_{buffer_dist}_km"]].rename(
            columns={f"glob_{buffer_dist}_km": "geometry"}
        )
        gdf_res.to_file(f"../../data/PREDICTS/site_coord_buff_{buffer_dist}_km.shp")

In [109]:
def get_raster_stats(polygon_path, raster_path, metric="mean", include_all_pixels=True):

    # Calculate zonal statistics
    stats = zonal_stats(
        vectors=polygon_path,
        raster=raster_path,
        stats=metric,
        all_touched=include_all_pixels,
    )

    # Get values and add columns to original dataframe
    result = np.array([x[metric] for x in stats])

    return result

In [94]:
# Set buffer distances
buffer_dist_km = [1, 10, 25, 50]

## Project, buffer and reproject sampling location coordinates

In [95]:
gdf_sites = gpd.read_file("../../data/PREDICTS/site_coord.shp")
gdf_sites.head()

,SSS,geometry
0,AD1_2001__Liow 1 1,POINT (103.77861 1.35194)
1,AD1_2001__Liow 1 2,POINT (103.80806 1.35472)
2,AD1_2001__Liow 1 3,POINT (103.81167 1.39472)
3,AD1_2001__Liow 1 4,POINT (103.78722 1.32694)
4,AD1_2001__Liow 1 5,POINT (103.80361 1.28278)


In [96]:
# Generate buffered polygons for sampling sites
start = time.time()

gdf_sites_buff = project_buffer_reproject_sites(gdf_sites, buffer_dist_km)

end = time.time()
print(end - start)

341.6535003185272


In [98]:
# Save separate file for each buffer radius
save_buffered_site_files(gdf_sites_buff, buffer_dist_km)

In [97]:
gdf_sites_buff.head()

,SSS,global_coord,utm_coord,epsg_code,utm_1_km,utm_10_km,utm_25_km,utm_50_km,glob_1_km,glob_10_km,glob_25_km,glob_50_km
0,AD1_2001__Liow 1 1,POINT (103.77861 1.35194),POINT (364117.2270177193 149464.9354869793),EPSG:32648,"POLYGON ((365117.2270177193 150464.9354869793,...","POLYGON ((374117.2270177193 159464.9354869793,...","POLYGON ((389117.2270177193 174464.9354869793,...","POLYGON ((414117.2270177193 199464.9354869793,...","POLYGON ((103.78759 1.36099, 103.78760 1.34290...","POLYGON ((103.86844 1.44244, 103.86852 1.26153...","POLYGON ((104.00320 1.57819, 104.00338 1.12590...","POLYGON ((104.22784 1.80445, 104.22812 0.89981..."
1,AD1_2001__Liow 1 2,POINT (103.80806 1.35472),POINT (367393.6320973808 149770.4037510926),EPSG:32648,"POLYGON ((368393.6320973808 150770.4037510926,...","POLYGON ((377393.6320973808 159770.4037510926,...","POLYGON ((392393.6320973808 174770.4037510926,...","POLYGON ((417393.6320973808 199770.4037510926,...","POLYGON ((103.81704 1.36377, 103.81705 1.34568...","POLYGON ((103.89789 1.44522, 103.89797 1.26431...","POLYGON ((104.03265 1.58097, 104.03283 1.12867...","POLYGON ((104.25729 1.80723, 104.25757 0.90258..."
2,AD1_2001__Liow 1 3,POINT (103.81167 1.39472),POINT (367797.6346832808 154192.390200358),EPSG:32648,"POLYGON ((368797.6346832808 155192.390200358, ...","POLYGON ((377797.6346832808 164192.390200358, ...","POLYGON ((392797.6346832808 179192.390200358, ...","POLYGON ((417797.6346832808 204192.390200358, ...","POLYGON ((103.82065 1.40377, 103.82066 1.38568...","POLYGON ((103.90150 1.48522, 103.90158 1.30431...","POLYGON ((104.03626 1.62097, 104.03645 1.16868...","POLYGON ((104.26091 1.84723, 104.26119 0.94258..."
3,AD1_2001__Liow 1 4,POINT (103.78722 1.32694),POINT (365074.0127648303 146700.5662553131),EPSG:32648,"POLYGON ((366074.0127648303 147700.5662553131,...","POLYGON ((375074.0127648303 156700.5662553131,...","POLYGON ((390074.0127648303 171700.5662553131,...","POLYGON ((415074.0127648303 196700.5662553131,...","POLYGON ((103.79620 1.33599, 103.79621 1.31790...","POLYGON ((103.87705 1.41744, 103.87713 1.23653...","POLYGON ((104.01181 1.55319, 104.01199 1.10090...","POLYGON ((104.23645 1.77945, 104.23673 0.87480..."
4,AD1_2001__Liow 1 5,POINT (103.80361 1.28278),POINT (366895.2831269228 141816.8386431553),EPSG:32648,"POLYGON ((367895.2831269228 142816.8386431553,...","POLYGON ((376895.2831269228 151816.8386431553,...","POLYGON ((391895.2831269228 166816.8386431553,...","POLYGON ((416895.2831269228 191816.8386431553,...","POLYGON ((103.81259 1.29183, 103.81260 1.27374...","POLYGON ((103.89344 1.37327, 103.89352 1.19236...","POLYGON ((104.02820 1.50902, 104.02837 1.05672...","POLYGON ((104.25284 1.73527, 104.25310 0.83062..."


## Extract population density data from Gridded Population of the World (GPW), v4

https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals-rev11

Unit: Number of people per square kilometer.

In [107]:
pop_density_paths = [
    "../../data/GPW/gpw_v4_2000_30_sec.tif",
    "../../data/GPW/gpw_v4_2005_30_sec.tif",
    "../../data/GPW/gpw_v4_2010_30_sec.tif",
    "../../data/GPW/gpw_v4_2015_30_sec.tif",
    "../../data/GPW/gpw_v4_2020_30_sec.tif",
]

In [108]:
pgp.get_raster_info(pop_density_paths[0])

{'file_list': ['../../data/GPW/gpw_v4_2000_30_sec.tif'],
 'projection_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
 'geotransform': (-180.0,
  0.00833333333333387,
  0.0,
  90.00000000001157,
  0.0,
  -0.00833333333333387),
 'pixel_size': (0.00833333333333387, -0.00833333333333387),
 'raster_size': (43200, 21600),
 'n_bands': 1,
 'nodata': [-3.4028230607370965e+38],
 'overviews': [],
 'block_size': [43200, 1],
 'bounding_box': [-180.0, -90.0, 180.00000000002314, 90.00000000001157],
 'datatype': 6,
 'numpy_type': numpy.float32}

In [103]:
# Extract population data and store in dataframe
start = time.time()

gdf_pop_density = gdf_sites.copy()

for buffer_dist in buffer_dist_km:
    for raster_path in pop_density_paths:
        get_raster_stats(

end = time.time()
print(end - start)

90.0772430896759


In [ ]:
vectors="../../data/PREDICTS/site_coord_buff_1_km.shp",
        raster="../../data/GPW/gpw_v4_2000_30_sec.tif",

In [104]:
gdf_pop_density

,SSS,geometry,Pop_density_2000_1km
0,AD1_2001__Liow 1 1,POINT (103.77861 1.35194),8665.053819
1,AD1_2001__Liow 1 2,POINT (103.80806 1.35472),1131.090088
2,AD1_2001__Liow 1 3,POINT (103.81167 1.39472),4979.729167
3,AD1_2001__Liow 1 4,POINT (103.78722 1.32694),4332.883681
4,AD1_2001__Liow 1 5,POINT (103.80361 1.28278),8465.158203
...,...,...,...
35731,YY1_2018__Guillemot 1 66,POINT (75.48022 12.19213),590.265340
35732,YY1_2018__Guillemot 1 67,POINT (75.48012 12.21315),590.265299
35733,YY1_2018__Guillemot 1 7,POINT (75.52462 12.26442),193.610229
35734,YY1_2018__Guillemot 1 8,POINT (75.52417 12.26392),193.610229


## Global Roads Open Access Data Set (gROADS), v1

https://sedac.ciesin.columbia.edu/data/set/groads-global-roads-open-access-v1

One data set available for each continent.

### Metadata description

- `OBJECTID`: Object ID
- `SourceID`: Source ID
- `Picture`: Picture
- `Exs`: Existence Category. Options include 1=Definite, 2=Doubtful, 0=Unspecified
- `Notes`: Notes
- `RoadID`: Road ID
- `ONme`: Official Road Name
- `RteNme`: Route Name
- `NtlClass`: National Inventory Road Class
- `FClass`: Functional Class with options 1=Highway, 2=Primary, 3=Secondary, 4=Tertiary, 5=Local/ Urban, 6=Trail, 7=Private, 0=Unspecified
- `Crgway`: Carriageways. Options include 1=Single, 2=Dual, 0=Unspecified
- `NumLanes`: Number of lanes
- `LneWidthM`: Lane Width in meters
- `RdWidthM`: Road Width in meters
- `AxleLoadMT`: Maximum Axle Loading in MT
- `TotLoadMT`: Maximum Total Loading in MT
- `SrfTpe`: Surface Type with options 1=Paved, 2=Gravel, 3=Dirt/Sand, 4=Steel, 5=Wood, 6=Grass, 0=Unspecified
- `SrfCond`: Surface Condition with options 1=Rough (<40kph), 2=Smooth (>40kph), 3=Snow/Ice, 4=Mud, 0=Unspecified
- `SrfPrep`: Surface Preparation with options 1=Natural Compaction, 2=Traffic Compaction, 3=Engineered Compaction, 4=Uncompacted, 0=Unspecified
- `IsSeasonal`: Affected by Season. Options include 1=Yes, 2=No, 0=Unspecified
- `CurntPrac`: Current Road Practicability with options 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `GdWthrPrac`: Good Weather Road Practicability with options 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `BdWthrPrac`: Bad Weather Road Practicability. Options include 1=Non-motorized, 2=Motorbike, 3=4WD <3.5MT, 4=Light Truck <10MT, 5=Heavy Truck <20MT, 6=Truck + Trailer >20MT, 0=Unspecified
- `SpeedLimit`: Speed Limit in Km/hr
- `CurntSpeed`: Current Average Speed
- `GnralSpeed`: General Average Speed
- `IsUndrCstr`: Is under Construction / Repairs. Options include 1=Yes, 2=No, 0=Unspecified
- `CstWrkETC`: Construction Work Estimated Completion Date
- `GradDeg`: Gradient in degrees
- `Sec`: Road Security Category. Options include 1=Category A (low risk), 2=Category B (low to medium risk), 3=Category C (medium to high risk), 4=Category D (high risk), 5=Category E (critical risk), 0=Unspecified
- `HasShouldr`: Has Shoulder. Options include 1=Yes, 2=No, 0=Unspecified
- `HasSidewalk`: Has Sidewalk. Options include 1=Yes, 2=No, 0=Unspecified
- `DrivSide`: Driving Side. Options include 1=Left, 2=Right, 0=Unspecified
- `IsElevated`: Is elevated / suspended above ground/water. Options include 1=Yes, 2=No, 0=Unspecified
- `HasMedian`: Has Median. Options include 1=Yes, 2=No, 0=Unspecified
- `OpStatus`: Operational Status. Options include 1=Open, 2=Restricted, 3=Closed, 4=Abandoned/Disused, 0=Unspecified
- `Shape_Length`: Length of segment
- `Length_KM`: Length of segment in kilometers



In [93]:
roads_americas = "../../data/gROADS/americas/groads-v1-americas.shp"
# groads_africa =
# groads_asia =
# groads_europe =
# groads_oceania_east =
# groads_oceania_west =

In [94]:
gdf_roads = gpd.read_file(roads_americas)
gdf_roads.head(2)

,SOURCEID,EXS,NOTES,ROADID,ONME,RTENME,NTLCLASS,FCLASS,CRGWAY,NUMLANES,LNEWIDTHM,RDWIDTHM,AXLELOADMT,TOTLOADMT,SRFTPE,SRFCOND,SRFPREP,ISSEASONAL,CURNTPRAC,GDWTHRPRAC,BDWTHRPRAC,SPEEDLIMIT,CURNTSPEED,GNRALSPEED,ISUNDRCSTR,CSTWRKETC,GRADDEG,SEC,HASSHOULDR,HASSIDEWLK,DRIVSIDE,ISELEVATED,HASMEDIAN,OPSTATUS,LENGTH_KM,Shape_Leng,geometry
0,s034_0001,0.0,None,ITOS156229,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.086124,0.216327,"LINESTRING (-46.64422 -22.52663, -46.63456 -22..."
1,s034_0001,0.0,None,ITOS156230,None,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.525002,0.338829,"LINESTRING (-49.61825 -23.06787, -49.54568 -23..."


In [95]:
print(gdf_roads.crs)

EPSG:4326


## WorldClim elevation data

https://worldclim.org/data/worldclim21.html#google_vignette

Same structure and resolution as the population density data. Can use the same approach to extract values for buffered sampling locations.

In [89]:
elevation_path = "../../data/WorldClim/Elevation/wc2.1_30s_elev.tif"
pgp.get_raster_info(elevation_path)

{'file_list': ['../../data/WorldClim/Elevation/wc2.1_30s_elev.tif'],
 'projection_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
 'geotransform': (-180.0,
  0.008333333333333333,
  0.0,
  90.0,
  0.0,
  -0.008333333333333333),
 'pixel_size': (0.008333333333333333, -0.008333333333333333),
 'raster_size': (43200, 21600),
 'n_bands': 1,
 'nodata': [-32768.0],
 'overviews': [],
 'block_size': [43200, 1],
 'bounding_box': [-180.0, -90.0, 180.0, 90.0],
 'datatype': 3,
 'numpy_type': numpy.int16}

## WorldClim bioclimatic data

https://worldclim.org/data/bioclim.html

In [96]:
bioclim_path = "../../data/WorldClim/Bioclimatic/wc2.1_30s_bio_1.tif"
pgp.get_raster_info(bioclim_path)

{'file_list': ['../../data/WorldClim/Bioclimatic/wc2.1_30s_bio_1.tif'],
 'projection_wkt': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
 'geotransform': (-180.0,
  0.008333333333333333,
  0.0,
  90.0,
  0.0,
  -0.008333333333333333),
 'pixel_size': (0.008333333333333333, -0.008333333333333333),
 'raster_size': (43200, 21600),
 'n_bands': 1,
 'nodata': [-3.3999999521443642e+38],
 'overviews': [],
 'block_size': [43200, 1],
 'bounding_box': [-180.0, -90.0, 180.0, 90.0],
 'datatype': 6,
 'numpy_type': numpy.float32}

## Unused code

In [ ]:
downscale_factor = 10  # Go from 1km to 10km

with rasterio.open("../../data/GPW/gpw_v4_2000_30_sec.tif") as dataset:

    # Read dataset and specify output shape
    # Use the average population density of the pixels being aggregated
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height / downscale_factor),
            int(dataset.width / downscale_factor),
        ),
        resampling=Resampling.average,
    )

    # Scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]), (dataset.height / data.shape[-2])
    )

    # Define the metadata for the new downsampled raster
    out_meta = dataset.meta.copy()
    out_meta.update(
        {
            "driver": "GTiff",
            "height": data.shape[1],
            "width": data.shape[2],
            "transform": transform,
        }
    )

    # Write the downsampled raster to a new file
    with rasterio.open("../../data/GPW/gpw_v4_2000_10_km.tif", "w", **out_meta) as dest:
        dest.write(data)

In [ ]:
start = time.time()

# Load the raster file
raster_data = rasterio.open("../../data/GPW/gpw_v4_2000_10_km.tif")

# Find the value used for nodata
nodata_value = raster_data.nodatavals[0]

# Extract population density from the grids for each coordinate
pop_density = np.array(list(raster_data.sample(coord_list))).flatten()

# Set density to np.nan if it seems to be a nodata pixel
if nodata_value is not None:
    pop_density[np.isclose(pop_density, nodata_value, equal_nan=True)] = np.nan

# Append new column to the dataframe
gdf_sites_point = gdf_sites_point.copy()
gdf_sites_point["Pop_density_2000_10km"] = pop_density

end = time.time()
print(end - start)

### Method 1: Rasterio sample

Get data for the grid in which the coordinates fall. This can work for high resolution data, but might be inaccurate when downsampling the population density data and then matching pixels with the site coordinates.

In [ ]:
start = time.time()

# Load the raster file
raster_data = rasterio.open("../../data/GPW/gpw_v4_2000_30_sec.tif")

# Find the value used for nodata
nodata_value = raster_data.nodatavals[0]

# Extract population density from the grids for each coordinate
pop_density = np.array(list(raster_data.sample(coord_list))).flatten()

# Set density to np.nan if it seems to be a nodata pixel
if nodata_value is not None:
    pop_density[np.isclose(pop_density, nodata_value, equal_nan=True)] = np.nan

# Append new column to the dataframe
gdf_sites_point = gdf_sites.copy()
gdf_sites_point["Pop_density_2000_1km"] = pop_density

end = time.time()
print(end - start)